In [78]:
import requests
import json
import openai

# JSON URL
url = "https://raw.githubusercontent.com/NomaDamas/KICE_slayer_AI_Korean/refs/heads/master/data/2023_11_KICE.json"

# JSON 데이터 불러오기
response = requests.get(url)
data = response.json()

# OpenAI API Key 설정
openai.api_key = "~~~"

# GPT-4 모델 설정
model = "gpt-4o"

In [82]:
# 지문을 가져오는 함수
def get_paragraph(id):
    for item in data:
        if item["id"] == id:
            return item["paragraph"]
    return None

# 문제 예측 함수
def prediction(problem, paragraph):
    question = problem["question"]
    choices = problem["choices"]
    
    # GPT-4에게 질문과 선택지를 제공하여 예측을 받음
    messages = [
        {"role": "system", "content": "당신은 언어 천재 AI 입니다"},
        {"role": "user", "content": f"다음 지문을 읽고, 질문에 답해주세요.\n지문: {paragraph}\n문제: {question}\n선택지: {', '.join(choices)}\n\n답만 말하고 이유는 말하지마. 만약 여러개의 답을 고르라하지 않으면 하나만 choice 해주세요. 또한 답을 말할 때 선택한 choice의 문장만 딱 말해주세요. choice에 없는 문장을 답으로 말하지 마세요."}
    ]
    
    # GPT-4 모델로 답 예측
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        max_tokens=100,  # 충분히 긴 답변을 받기 위한 토큰 수 증가
        temperature=0.5
    )
    
    # 모델의 응답에서 예측된 답과 그 이유를 추출
    answer_and_reason = response['choices'][0]['message']['content'].strip()
    
    # 예측된 답을 답과 이유로 분리
    if "\n" in answer_and_reason:
        answer, reason = answer_and_reason.split("\n", 1)
    else:
        answer = answer_and_reason
        reason = "이유가 제공되지 않았습니다."
    
    print(f"답 : {answer}\n")
    print(f"{reason}\n")
    
    # 예측된 답을 선택지 번호로 변환
    for idx, choice in enumerate(choices, 1):
        print(f"비교 중: 선택지 {idx}: {choice}")  # 각 선택지를 출력
        if choice.strip().lower() in answer.strip().lower():  # 대소문자 및 공백 차이 무시
            return idx  # 선택된 답을 번호로 반환
    return 3  # 답을 찾지 못한 경우 3번으로 찍음
    
# 채점 함수
def calculate_score(data):
    total_score = 0
    total_questions = 0
    
    for item in data:
        paragraph_id = item.get("id", "")  # 문제의 ID를 가져와서 해당 지문을 찾음
        paragraph = get_paragraph(paragraph_id)  # 해당 ID에 맞는 paragraph를 가져옴
        for problem in item.get("problems", []):  # "problems"가 없을 경우 대비
            total_questions += 1
            predicted_answer = prediction(problem, paragraph)
            correct_answer = problem["answer"]
            
            if predicted_answer == correct_answer :
                total_score += problem.get("score", 0)  # 점수가 없을 경우 기본값 0
            
            # 문제와 예측 결과 출력
            print(f"문제: {problem['question']}")
            print(f"정답: {correct_answer}")            
            print(f"예측된 답: {predicted_answer}")
            print(f"점수: {problem.get('score', 0)}")
            print("---")
    
    return total_score, total_questions

# 점수 계산 실행
total_score, total_questions = calculate_score(data)

답 : 독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다.

이유가 제공되지 않았습니다.

비교 중: 선택지 1: 같은 책을 읽은 독자라도 서로 다른 의미를 구성할 수 있다.
비교 중: 선택지 2: 다른 독자와의 소통은 독자가 인식의 폭을 확장하도록 돕는다
비교 중: 선택지 3: 독자는 직접 경험해 보지 못했던 다양한 삶을 책의 필자를 매개로 접할 수 있다.
비교 중: 선택지 4: 독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다.
문제: 윗글의 내용과 일치하지 않는 것은?
정답: 4
예측된 답: 4
점수: 2
---
답 : ⓐ에는 ‘책에서 답을 찾는 질문’이, ⓔ에는 그에 대한 답을 ‘독자의 경험’에서 찾아내는 모습이 나타난다.

이유가 제공되지 않았습니다.

비교 중: 선택지 1: ⓐ와 ⓑ에는 모두 ‘독자 자신에게서 답을 찾는 질문’이 나타난다.
비교 중: 선택지 2: ⓒ와 ⓓ에는 모두 ‘책에 명시된 내용’에서 질문의 답을 찾아내는 모습이 나타난다.
비교 중: 선택지 3: ⓐ에는 ‘책에서 답을 찾는 질문’이, ⓔ에는 그에 대한 답을 ‘독자의 경험’에서 찾아내는 모습이 나타난다.
문제: 다음은 학생이 독서 후 작성한 글의 일부이다. [A]를 바탕으로 ⓐ～ⓔ를 이해한 내용으로 가장 적절한 것은?
정답: 5
예측된 답: 3
점수: 3
---
답 : 스스로 독서 계획을 세우고 자신에게 필요한 책을 찾아 개인적으로 읽는 과정에서 경험할 수 있겠군.

이유가 제공되지 않았습니다.

비교 중: 선택지 1: 스스로 독서 계획을 세우고 자신에게 필요한 책을 찾아 개인적으로 읽는 과정에서 경험할 수 있겠군.
문제: 윗글을 읽고 ㉠에 대해 보인 반응으로 적절하지 않은 것은?
정답: 1
예측된 답: 1
점수: 2
---
답 : (가)는 유서의 특성과 의의를 설명하였고, (나)는 유서 편찬에서 특정 학문의 수용 양상을 시기별로 소개하였다.

이유가 제공되지 않았습니다.

비교 중: 선택지 1: (가

In [84]:

# 결과 출력
print(f"총 점수: {total_score} / 총 문제 수: {total_questions}")
print(f"GPT-4 점수: {(total_score / (total_questions * 2)) * 100}%")


총 점수: 81 / 총 문제 수: 45
GPT-4 점수: 90.0%
